# ニューラルネットワークモデルの作り方

In [1]:
%matplotlib inline

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
#訓練に使用するデバイス
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cuda device


In [4]:
#クラスの定義
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__() #super()は「親クラスを参照するための関数」
        self.flatten = nn.Flatten() #バッチ次元は残して、それ以降の次元を全部くっつけて1次元に
        self.linear_relu_stack = nn.Sequential( #複数のレイヤーを順番に実行するためのコンテナ
            nn.Linear(28*28, 512),              #Sequential = 「順番に」「連続して」
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    
    def forward(self, x):
        x = self.flatten(x) #nn.Flatten()を格納した変数なので引数を受け取れる
        logits = self.linear_relu_stack(x) #logitsは「活性化関数をかける前の生の線形出力」
        return logits

## `super().__init__()` がないとどうなるか

### **1. 結論**
***`super().__init__()` を呼ばないと、PyTorchモデルとして正常に動作せず学習できない。***

---

### **2. 理由**
`nn.Module` は内部で以下の管理を行っている。

- **パラメータ辞書 (`_parameters`)**  
  モデル内の学習対象パラメータを登録する
- **サブモジュール辞書 (`_modules`)**  
  モデル内のレイヤー（`nn.Linear` など）を登録する
- **バッファ辞書 (`_buffers`)**  
  BatchNormの平均値や分散など、学習しないが保存する値を登録する

***`super().__init__()` を呼ばないと、これらの辞書が初期化されないため、パラメータやモジュールが全く管理されない。***

---

### **3. 実際に起こる問題**

1. **`.parameters()` が空になる**  
   → Optimizerに渡すパラメータがない → ***重みが更新されない***

2. **`.to(device)` が効かない**  
   → GPUに移動できない

3. **`.train()` / `.eval()` が効かない**  
   → DropoutやBatchNormの挙動が切り替わらない

4. **`state_dict()` が空**  
   → ***学習済みモデルの保存・読み込みができない***

---

### **4. 実験コード例**
```python
import torch
from torch import nn

# super().__init__() を呼ばない悪い例
class BadModel(nn.Module):
    def __init__(self):
        # super().__init__() がない！
        self.fc = nn.Linear(10, 1)

bad_model = BadModel()
print("BadModel parameters:", list(bad_model.parameters()))  # → []

# 正しい例
class GoodModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(10, 1)

good_model = GoodModel()
print("GoodModel parameters:", list(good_model.parameters()))  # → fcのパラメータが表示される


## ダンダーメソッド（マジックメソッド）まとめ

### **1. ダンダーメソッドとは**
- **d**ouble **under**score → **dunder method**
- 名前が `__◯◯__` の形式になっている特別なメソッド
- Pythonが特定のタイミングで **自動的に呼び出す**
- 例：
  - `__init__` → インスタンス生成直後に呼ばれる
  - `__len__` → `len(obj)` を呼んだときに実行される
  - `__getitem__` → `obj[key]` と書いたときに実行される

---

### **2. 基本的な特徴**
- **自分で直接呼び出すことは可能**だが、通常は構文や組み込み関数を通して使う
- 名前を `__` で囲むのは以下の理由：
  - 通常のメソッドと区別するため
  - 名前の衝突を避けるため
  - 「ユーザーが直接呼ぶものではない」というサイン

---

### **3. よく使うダンダーメソッド一覧**

| メソッド         | いつ呼ばれるか                     | 使用例 |
|------------------|-----------------------------------|--------|
| `__init__`       | インスタンス生成時                 | `obj = MyClass()` |
| `__call__`       | 関数のように呼び出されたとき       | `obj()` |
| `__len__`        | `len(obj)`                        | `len(my_obj)` |
| `__getitem__`    | インデックスアクセス時             | `obj[i]` |
| `__setitem__`    | インデックス代入時                 | `obj[i] = x` |
| `__iter__`       | イテレーション開始時               | `for x in obj:` |
| `__str__`        | `str(obj)` / `print(obj)`          | `print(my_obj)` |

---

### **4. 実例コード**
```python
class MyList:
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

    def __str__(self):
        return f"MyList({self.data})"

ml = MyList([1, 2, 3]) #この引数はinitの引数(data)に代入される
print(len(ml))      # __len__ が呼ばれる → len(data)=3
print(ml[1])        # インデックスアクセスにより、__getitem__ が呼ばれる → data[1]=2
print(ml)           # printはオブジェクトを文字列化するので、__str__ が呼ばれる → MyList([1, 2, 3])


In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [6]:
X = torch.rand(1, 28, 28, device=device) #この1はチャネル数ではなくバッチサイズ（として解釈される）
logits = model(X) #logitsは(batch_size, num_classes)。今回は(1, 10)
softmax = nn.Softmax(dim=1) #Softmaxクラス（nn.Moduleのサブクラス）からオブジェクトを作成。dim=1は__init__の引数
pred_probab = softmax(logits) #dim=1によりそれぞれのクラスの数値にsoftmaxが適用→ただの数値が確率(probability)に
y_pred = pred_probab.argmax(dim=1) #argmax()は最大値のインデックスを返す関数。maxは数値を返す
print(f"Predict class: {y_pred}")

Predict class: tensor([8], device='cuda:0')


## `forward()` を定義する理由
PyTorch の `nn.Module` を継承したクラスでは、モデルの順伝播処理（入力 → 出力）を `forward()` に書く必要がある。  
`forward()` の中で、入力テンソルがどのように変換されるかを定義する。

例：
- `nn.Flatten()` で `28x28` の画像を 1 次元ベクトルに変換
- `nn.Sequential` で複数の線形層と活性化関数を順に適用
- 最終的に活性化関数をかける前の出力（`logits`）を返す

`forward()` を定義しないと、モデルがどのように計算されるかが決まらないため、必ず実装する必要がある。

---

## `forward()` を直接呼ばない理由
`forward()` を直接呼び出すと、`nn.Module.__call__()` が行う以下の処理がスキップされる。
- `train()` / `eval()` 状態に応じた挙動の切り替え
- 自動微分用の勾配追跡開始
- `forward()` 前後で登録されたフック処理の実行

これらは学習や推論の正しい動作に必要なため、`forward()` を直接呼ぶのは非推奨。

---

## 正しい呼び出し方
モデルを呼び出すときは、必ず `model(X)` の形にする。  
`model(X)` は内部で `__call__()` が実行され、必要な背景処理を行った上で 自分で定義した`forward(X)` が呼ばれる。


In [7]:
#FashionMNISTモデルを各レイヤーレベルで確認
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [8]:
#nn.Flatten
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [9]:
#nn.Linear
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [10]:
#nn.ReLU
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1) #分けずに一括でやってもいい
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.5859, -0.4630, -0.2106, -0.2163, -0.1700, -0.3592,  0.0604, -0.0669,
         -0.1404, -0.0897, -0.1948, -0.1026,  0.0585,  0.2663, -0.8166, -0.2806,
          0.3159, -0.0583,  0.5394,  0.3058],
        [ 0.3369,  0.0271, -0.1892, -0.0128, -0.2601, -0.2365, -0.2626,  0.2252,
         -0.1082,  0.1773, -0.0093,  0.0255, -0.3024,  0.3794, -0.8594, -0.1339,
          0.1560,  0.0070,  0.5540,  0.5493],
        [ 0.4989, -0.5179, -0.3275, -0.1282, -0.1387, -0.1049, -0.0212,  0.2768,
         -0.0163,  0.1466,  0.1595,  0.0123, -0.0054,  0.2037, -0.3283,  0.3755,
          0.3095,  0.0384,  0.6014,  0.6277]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.5859, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0604, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0585, 0.2663, 0.0000, 0.0000, 0.3159, 0.0000,
         0.5394, 0.3058],
        [0.3369, 0.0271, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2252, 0.0000,
         0.1773, 0.0000, 0.0255, 0.0000, 0.3794, 0.00

In [11]:
#nn.Sequential
seq_models = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_models(input_image)

In [12]:
#nn.Softmax
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [13]:
#モデルパラメータ
#nn.Module を継承することで、モデルオブジェクト内で定義されたすべてのフィールドが自動的に追跡でき、
#parameters() や named_parameters() メソッドを使って、モデルの各レイヤーのすべてのパラメータにアクセスできるようになる

print("Model Structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model Structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[-0.0012, -0.0010,  0.0088,  ...,  0.0281,  0.0008, -0.0099],
        [-0.0186, -0.0079,  0.0236,  ..., -0.0207, -0.0353, -0.0142]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([ 0.0350, -0.0311], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[ 0.0385,  0.0047,  0.0317,  ...,  0.0218, -0.0332, -0.0024],
        [-0.0036, -0.0271,  0.0302,  ...,  0.0436, -0.0134, -0.0315]],
       device='cuda: